In [ ]:
import json
import requests

## How to create a Twitter keyword monitor with Python and Slack notifications

This reply sparked an idea to build a keyword monitoring tool for Twitter with Slack notifications whenever someone tweets anything with the keywords I'm monitoring ⬇️

![Monitoring tool idea](https://d2pwmb8xsybju4.cloudfront.net/posts/create-keyword-monitoring-in-twitter-with-slack-notifications/0_idea_cropped.png "Monitoring tool idea")

The reply is to this awesome Indie Hackers post [$0-$20K in 2 months,
here's tactics that worked](https://www.indiehackers.com/post/0-20k-in-2-months-heres-tactics-that-worked-b4d49794f6 "$0 - $20K in 2 months,
here's tactics that worked")

**In this walkthrough, we'll build a social media listener for indie hackers looking to shamelessly promote what they're making by monitoring the keyword:**

Detailed step-by-step intructions for this repo in this blog post: https://norahsakal.com/blog/create-keyword-monitoring-in-twitter-with-python-with-slack-notifications

# Fetch tweets with your keywords

In [ ]:
bearer_token = "YOUR_BEARER_TOKEN"

search_url = "https://api.twitter.com/2/tweets/search/recent"

search_headers = {
    'Authorization': f'Bearer {bearer_token}' 
}
search_resp = requests.get(
    search_url,
    headers=search_headers, 
    params={
        "query":'"what are you building" -is:retweet -is:reply OR "what are you working on" -is:retweet -is:reply OR "anyone building cool things" -is:retweet -is:reply',
        'tweet.fields':'author_id,public_metrics,created_at,entities',
        'expansions':'author_id',
        'user.fields':'username,description,public_metrics,profile_image_url',
        "max_results":20 # Optional
    }
)
search_resp.json()

# Create dict with data from each tweet

In [ ]:
tweet_summary = []
for i,tweet in enumerate(search_resp.json()['data']):
    
    temp_dict = {}
    
    # Tweet data
    temp_dict['tweet'] = tweet['text']
    temp_dict['created_at'] = tweet['created_at']
    temp_dict['retweet_count'] = tweet['public_metrics']['retweet_count']
    temp_dict['reply_count'] = tweet['public_metrics']['reply_count']
    temp_dict['like_count'] = tweet['public_metrics']['like_count']
    temp_dict['quote_count'] = tweet['public_metrics']['quote_count']
    temp_dict['tweet_url'] = f"https://twitter.com/{search_resp.json()['includes']['users'][i]['username']}/status/{tweet['id']}"
    
    # User data
    temp_dict['user_description'] = search_resp.json()['includes']['users'][i]['description']
    temp_dict['name'] = search_resp.json()['includes']['users'][i]['name']
    temp_dict['username'] = search_resp.json()['includes']['users'][i]['username']
    temp_dict['followers_count'] = search_resp.json()['includes']['users'][i]['public_metrics']['followers_count']
    temp_dict['following_count'] = search_resp.json()['includes']['users'][i]['public_metrics']['following_count']
    temp_dict['tweet_count'] = search_resp.json()['includes']['users'][i]['public_metrics']['tweet_count']
    temp_dict['profile_img'] = search_resp.json()['includes']['users'][i]['profile_image_url']
    
    # Check if hashtags in tweet
    if 'entities' in tweet:
        if 'hashtags' in tweet['entities']:
            tweet_hashtags = []
            for hashtag in tweet['entities']['hashtags']:
                tweet_hashtags.append(hashtag['tag'])
            temp_dict['tweet_hashtags'] = tweet_hashtags
    
    tweet_summary.append(temp_dict)
    
tweet_summary


# Send tweets to Slack

In [ ]:
slack_webhook_url = "YOUR_SLACK_WEBHOOK_URL"

for tweet in tweet_summary:
    temp_tweet = tweet['tweet'].replace('\n', ' ')  # Optional to remove \n
    slack_data = {
        "icon_emoji": ':mega:',
        "text": f"Promote yourself here \n *tweet:* {temp_tweet} \n *Author:* {tweet['name']} \n *Description:* {tweet['user_description']} \n *URL:* {tweet['tweet_url']}",
        "username": "New promotion tweet 📣",
        "channel":'#integration-testing',
        "attachments": [
            {
                "fallback": "Required plain-text summary of the attachment.",
                "text": "Optional text that appears within the attachment",
                "image_url": f"{tweet['profile_img']}",
                "thumb_url": f"{tweet['profile_img']}",
            }
        ]
    }
    
    # Check if hashtag
    if 'tweet_hashtags' in tweet:
        slack_data['text'] += f"\n *Hashtags:* {', '.join(tweet['tweet_hashtags'])}" 
    
    response = requests.post(slack_webhook_url, data=json.dumps(slack_data),
                             headers={"Content-Type": "application/json"})
    print(response.text)